In [24]:
import csv
import datetime
from datetime import timedelta

import pandas as pd
import pyreadr
from matplotlib import pyplot as plt
from pygam import LinearGAM, s
from pygam.terms import TermList
from sklearn.linear_model import LinearRegression

output_cols = ['date_cible','perimetre_prod','type_prod','prev_FC']

In [4]:
substation_df = pd.read_csv('coord_substations.csv')
substation_df

,SUBSTATION,LON,LAT
0,A.COM,2.512180,50.272269
1,HOSTE,-0.625940,44.485958
2,CXSSE,4.511712,48.838806
3,ONDES,1.305962,43.784788


In [5]:
station_df = pd.read_csv('liste_stations.csv', sep=';')
station_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IOMM          35 non-null     int64 
 1   code_station  35 non-null     int64 
 2   Station       35 non-null     object
 3   NOM_USUEL     35 non-null     object
 4   Coeff         35 non-null     object
 5   Code_postal   35 non-null     int64 
 6   Code_insee    35 non-null     int64 
 7   Reg           35 non-null     object
 8   Label_reg     35 non-null     object
 9   Dep           35 non-null     int64 
 10  LATITUDE      35 non-null     object
 11  LONGITUDE     35 non-null     object
 12  ALTITUDE      35 non-null     int64 
dtypes: int64(6), object(7)
memory usage: 3.7+ KB


In [6]:
prod = pyreadr.read_r('prod.RDS')[None]
prod['time_in_day'] = prod.date_cible.apply(lambda dt: (3600*dt.hour+60*dt.minute + dt.second)/(24*3600))
train_prod = prod[prod.date_cible.dt.year < 2019]
test_prod = prod[prod.date_cible.dt.year == 2019]

train_prod

,date_cible,perimetre_prod,type_prod,comptage,puissance_installee,FC,time_in_day
0,2016-01-01 00:00:00,A.COM,eolien,47.500000,104.7,0.453677,0.000000
1,2016-01-01 00:00:00,CXSSE,eolien,24.200000,85.8,0.282051,0.000000
2,2016-01-01 00:00:00,HOSTE,pv,0.000000,54.0,0.000000,0.000000
3,2016-01-01 00:00:00,national,pv,0.000000,6006.0,0.000000,0.000000
4,2016-01-01 00:00:00,national,eolien,2199.000000,10201.0,0.215567,0.000000
...,...,...,...,...,...,...,...
315631,2018-12-31 23:30:00,CXSSE,eolien,0.166667,85.8,0.001943,0.979167
315632,2018-12-31 23:30:00,HOSTE,pv,0.000000,54.0,0.000000,0.979167
315633,2018-12-31 23:30:00,national,pv,0.000000,8328.0,0.000000,0.979167
315634,2018-12-31 23:30:00,national,eolien,1612.000000,14500.0,0.111172,0.979167


## Météo ECMWF

In [7]:
meteo_ce_ecmwf = pyreadr.read_r('meteo_ce_allvar_grille_ecmwf.RDS')[None]
meteo_ce_ecmwf.rename(columns={'zone':'perimetre_prod'},inplace=True)

def preprocess_meteo(df):
    # Keep valid previsions (date_lancement < veille, 16h)
    out_df = df[df.apply(keep_valid_forecasts, axis=1)]
    out_df.drop(columns='echeance',inplace=True)
    # Keep latest date_lancement for each date_cible
    out_df = out_df.groupby(['perimetre_prod', 'date_cible']).last().reset_index()

    return out_df

def keep_valid_forecasts(row):
    veille = (row.date_cible - timedelta(days=1))
    earliest_valid_lancement = veille.replace(hour=16)
    return row.date_lancement < earliest_valid_lancement


valid_ce_meteo_df = preprocess_meteo(meteo_ce_ecmwf)

/home/rozierale/.miniconda3/envs/origami/lib/python3.7/site-packages/pandas/core/frame.py:4908: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [8]:
# Check if only one date_lancement per date_cible
valid_ce_meteo_df.groupby(['perimetre_prod', 'date_cible']).count()

date_lancement  u100  v100  ff100  neb  \
perimetre_prod date_cible                                                    
France         2016-01-02 00:00:00               1     1     1      1    1   
               2016-01-02 00:30:00               1     1     1      1    1   
               2016-01-02 01:00:00               1     1     1      1    1   
               2016-01-02 01:30:00               1     1     1      1    1   
               2016-01-02 02:00:00               1     1     1      1    1   
...                                            ...   ...   ...    ...  ...   
SESO           2019-12-31 21:30:00               1     1     1      1    1   
               2019-12-31 22:00:00               1     1     1      1    1   
               2019-12-31 22:30:00               1     1     1      1    1   
               2019-12-31 23:00:00               1     1     1      1    1   
               2019-12-31 23:30:00               1     1     1      1    1   

                                    t2m  ssrd  
perimetre_prod date_cible                      
France         2016-01-02 00:00:00    1     1  
               2016-01-02 00:30:00    1     1  
               2016-01-02 01:00:00    1     1  
               2016-01-02 01:30:00    1     1  
               2016-01-02 02:00:00    1     1  
...                                 ...   ...  
SESO           2019-12-31 21:30:00    1     1  
               2019-12-31 22:00:00    1     1  
               2019-12-31 22:30:00    1     1  
               2019-12-31 23:00:00    1     1  
               2019-12-31 23:30:00    1     1  

[560640 rows x 7 columns]

In [9]:
meteo_locale_ecmwf = pyreadr.read_r('meteo_locale_allvar_grille_ecmwf.RDS')[None]
meteo_locale_ecmwf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 846504 entries, 0 to 846503
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   date_lancement  846504 non-null  datetime64[ns]
 1   perimetre_prod  846504 non-null  object        
 2   date_cible      846504 non-null  datetime64[ns]
 3   tcc             846504 non-null  float64       
 4   t2m             846504 non-null  float64       
 5   ssrd            846504 non-null  float64       
 6   u100            846504 non-null  float64       
 7   v100            846504 non-null  float64       
 8   ff100           846504 non-null  float64       
 9   echeance        846504 non-null  float64       
dtypes: datetime64[ns](2), float64(7), object(1)
memory usage: 64.6+ MB


In [10]:
valid_local_meteo_df = preprocess_meteo(meteo_locale_ecmwf)
valid_local_meteo_df

/home/rozierale/.miniconda3/envs/origami/lib/python3.7/site-packages/pandas/core/frame.py:4908: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,perimetre_prod,date_cible,date_lancement,tcc,t2m,ssrd,u100,v100,ff100
0,A.COM,2016-01-02 00:00:00,2016-01-01 12:00:00,1.000000,6.325450,2.216240e-03,-5.435037,9.539942,10.980536
1,A.COM,2016-01-02 00:30:00,2016-01-01 12:00:00,0.999551,6.344030,0.000000e+00,-5.078984,9.433556,10.707797
2,A.COM,2016-01-02 01:00:00,2016-01-01 12:00:00,1.000000,6.317258,-1.425500e-03,-4.810986,9.301626,10.473833
3,A.COM,2016-01-02 01:30:00,2016-01-01 12:00:00,1.000689,6.246803,-7.623297e-19,-4.448654,9.288761,10.308406
4,A.COM,2016-01-02 02:00:00,2016-01-01 12:00:00,1.000000,6.219141,3.485759e-03,-4.109660,9.396701,10.258342
...,...,...,...,...,...,...,...,...,...
280315,ONDES,2019-12-31 21:30:00,2019-12-30 12:00:00,0.589047,5.987815,0.000000e+00,2.682430,-0.612226,2.774608
280316,ONDES,2019-12-31 22:00:00,2019-12-30 12:00:00,0.623447,5.711979,8.224577e-03,2.946498,-0.379447,3.007768
280317,ONDES,2019-12-31 22:30:00,2019-12-30 12:00:00,0.679307,5.737721,0.000000e+00,3.167307,-0.073829,3.218369
280318,ONDES,2019-12-31 23:00:00,2019-12-30 12:00:00,0.730482,5.867768,-2.223821e-03,3.357581,0.190312,3.400280


## Météo France

In [11]:
meteo_ce_nebtemp_mf = pyreadr.read_r('meteo_ce_nebtemp_station_meteofrance.RDS')[None]
meteo_ce_nebtemp_mf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1655368 entries, 0 to 1655367
Data columns (total 6 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   date_lancement  1655368 non-null  datetime64[ns]
 1   zone            1655368 non-null  object        
 2   date_cible      1655368 non-null  datetime64[ns]
 3   t2m             1655368 non-null  float64       
 4   neb             1655368 non-null  float64       
 5   echeance        1655368 non-null  float64       
dtypes: datetime64[ns](2), float64(3), object(1)
memory usage: 75.8+ MB


In [12]:
meteo_ce_vent_mf = pyreadr.read_r('meteo_ce_vent_grille_meteofrance.RDS')[None]
meteo_ce_vent_mf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1403600 entries, 0 to 1403599
Data columns (total 7 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   date_lancement  1403600 non-null  datetime64[ns]
 1   zone            1403600 non-null  object        
 2   date_cible      1403600 non-null  datetime64[ns]
 3   u100            1403600 non-null  float64       
 4   v100            1403600 non-null  float64       
 5   ff100           1403600 non-null  float64       
 6   echeance        1403600 non-null  float64       
dtypes: datetime64[ns](2), float64(4), object(1)
memory usage: 75.0+ MB


In [13]:
meteo_locale_vent_mf = pyreadr.read_r('meteo_locale_vent_grille_meteofrance.RDS')[None]
meteo_locale_vent_mf.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 701800 entries, 0 to 701799
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   date_lancement  701800 non-null  datetime64[ns]
 1   perimetre_prod  701800 non-null  object        
 2   date_cible      701800 non-null  datetime64[ns]
 3   u100            701800 non-null  float64       
 4   v100            701800 non-null  float64       
 5   ff100           701800 non-null  float64       
 6   echeance        701800 non-null  float64       
dtypes: datetime64[ns](2), float64(4), object(1)
memory usage: 37.5+ MB


### Modèles nationaux



In [14]:
valid_meteo_df = valid_ce_meteo_df[valid_ce_meteo_df.perimetre_prod !='France'] # Remove france-level meteo
valid_meteo_df = valid_meteo_df.pivot_table(index=['date_cible'],columns='perimetre_prod')
valid_meteo_df

ff100                                             \
perimetre_prod            SEE       SENE       SENP        SEO     SERAA   
date_cible                                                                 
2016-01-02 00:00:00  8.319350  10.547959  11.542937  12.586564  9.531615   
2016-01-02 00:30:00  8.315284  10.298050  11.492712  12.766400  9.269736   
2016-01-02 01:00:00  8.305242  10.068043  11.498800  12.839232  8.954929   
2016-01-02 01:30:00  8.263552   9.882176  11.579808  12.779465  8.677415   
2016-01-02 02:00:00  8.184601   9.719266  11.704983  12.658765  8.412503   
...                       ...        ...        ...        ...       ...   
2019-12-31 21:30:00  5.537229   6.661943   5.350142   4.503309  2.484249   
2019-12-31 22:00:00  5.463723   6.592759   5.380939   4.620322  2.551837   
2019-12-31 22:30:00  5.368219   6.513374   5.334357   4.868887  2.595616   
2019-12-31 23:00:00  5.241389   6.447391   5.262606   5.069467  2.590055   
2019-12-31 23:30:00  5.092690   6.414585   5.224680   5.056874  2.530966   

                                              neb                      ...  \
perimetre_prod           SESE      SESO       SEE      SENE      SENP  ...   
date_cible                                                             ...   
2016-01-02 00:00:00  7.934719  6.646562  0.992722  1.000000  0.871739  ...   
2016-01-02 00:30:00  7.905147  6.145116  0.987574  1.000897  0.860362  ...   
2016-01-02 01:00:00  7.881863  5.591630  0.983167  0.999996  0.857950  ...   
2016-01-02 01:30:00  7.902884  5.094342  0.985177  0.999916  0.843299  ...   
2016-01-02 02:00:00  7.773933  4.709320  0.991122  1.000000  0.825116  ...   
...                       ...       ...       ...       ...       ...  ...   
2019-12-31 21:30:00  2.509494  4.805918  0.281775  0.128306  0.755680  ...   
2019-12-31 22:00:00  2.692023  4.870547  0.255661  0.129463  0.749536  ...   
2019-12-31 22:30:00  2.832131  5.035606  0.243020  0.071725  0.757069  ...   
2019-12-31 23:00:00  2.909110  5.275643  0.242331  0.035451  0.763265  ...   
2019-12-31 23:30:00  2.921020  5.533912  0.247831  0.086632  0.753688  ...   

                         u100                          v100            \
perimetre_prod          SERAA      SESE      SESO       SEE      SENE   
date_cible                                                              
2016-01-02 00:00:00 -0.996641 -3.069270 -3.742904  6.949960  8.418881   
2016-01-02 00:30:00 -0.709314 -3.306466 -2.923560  7.174743  8.426196   
2016-01-02 01:00:00 -0.380325 -3.616906 -1.996707  7.339470  8.461914   
2016-01-02 01:30:00 -0.120087 -3.800409 -0.955150  7.445176  8.532849   
2016-01-02 02:00:00  0.052006 -3.770372 -0.099607  7.482870  8.610320   
...                       ...       ...       ...       ...       ...   
2019-12-31 21:30:00  0.736390 -0.234154  3.768888  1.437880  4.069895   
2019-12-31 22:00:00  0.936919 -0.394858  3.830146  1.433428  4.333019   
2019-12-31 22:30:00  1.132665 -0.640280  3.953453  1.480148  4.427826   
2019-12-31 23:00:00  1.303951 -0.792954  4.106719  1.574605  4.441036   
2019-12-31 23:30:00  1.435879 -0.710795  4.240081  1.706246  4.469962   

                                                                         
perimetre_prod            SENP        SEO     SERAA      SESE      SESO  
date_cible                                                               
2016-01-02 00:00:00  10.842495  11.107833  8.983609  5.861311  2.381508  
2016-01-02 00:30:00  10.921941  11.009990  8.713750  5.673314  1.941363  
2016-01-02 01:00:00  11.053367  10.756720  8.429033  5.586181  1.346287  
2016-01-02 01:30:00  11.235311  10.373602  8.228635  5.647702  0.684354  
2016-01-02 02:00:00  11.427813   9.944114  8.040371  5.589341  0.305117  
...                        ...        ...       ...       ...       ...  
2019-12-31 21:30:00   4.448746   4.108127 -0.565819 -2.361213 -2.261000  
2019-12-31 22:00:00   4.555737   4.076441 -0.241374 -2.542925 -2.312492  
2019-12-31 22:30:00   4.63937

In [15]:
national_df = prod[prod.perimetre_prod=='national'].merge(valid_meteo_df, on='date_cible', how='inner')

national_df

/home/rozierale/.miniconda3/envs/origami/lib/python3.7/site-packages/pandas/core/frame.py:9199: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  validate=validate,


,date_cible,perimetre_prod,type_prod,comptage,puissance_installee,FC,time_in_day,"(ff100, SEE)","(ff100, SENE)","(ff100, SENP)",...,"(u100, SERAA)","(u100, SESE)","(u100, SESO)","(v100, SEE)","(v100, SENE)","(v100, SENP)","(v100, SEO)","(v100, SERAA)","(v100, SESE)","(v100, SESO)"
0,2016-01-02 00:00:00,national,pv,0.0,6006.0,0.000000,0.000000,8.319350,10.547959,11.542937,...,-0.996641,-3.069270,-3.742904,6.949960,8.418881,10.842495,11.107833,8.983609,5.861311,2.381508
1,2016-01-02 00:00:00,national,eolien,5543.0,10201.0,0.543378,0.000000,8.319350,10.547959,11.542937,...,-0.996641,-3.069270,-3.742904,6.949960,8.418881,10.842495,11.107833,8.983609,5.861311,2.381508
2,2016-01-02 00:30:00,national,pv,0.0,6006.0,0.000000,0.020833,8.315284,10.298050,11.492712,...,-0.709314,-3.306466,-2.923560,7.174743,8.426196,10.921941,11.009990,8.713750,5.673314,1.941363
3,2016-01-02 00:30:00,national,eolien,5335.0,10201.0,0.522988,0.020833,8.315284,10.298050,11.492712,...,-0.709314,-3.306466,-2.923560,7.174743,8.426196,10.921941,11.009990,8.713750,5.673314,1.941363
4,2016-01-02 01:00:00,national,pv,0.0,6006.0,0.000000,0.041667,8.305242,10.068043,11.498800,...,-0.380325,-3.616906,-1.996707,7.339470,8.461914,11.053367,10.756720,8.429033,5.586181,1.346287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140143,2019-12-31 22:30:00,national,eolien,NaN,16035.0,NaN,0.937500,5.368219,6.513374,5.334357,...,1.132665,-0.640280,3.953453,1.480148,4.427826,4.639371,4.051164,0.042446,-2.664343,-2.420645
140144,2019-12-31 23:00:00,national,pv,NaN,9258.0,NaN,0.958333,5.241389,6.447391,5.262606,...,1.303951,-0.792954,4.106719,1.574605,4.441036,4.709518,4.061788,0.207081,-2.724843,-2.573396
140145,2019-12-31 23:00:00,national,eolien,NaN,16035.0,NaN,0.958333,5.241389,6.447391,5.262606,...,1.303951,-0.792954,4.106719,1.574605,4.441036,4.709518,4.061788,0.207081,-2.724843,-2.573396
140146,2019-12-31 23:30:00,national,pv,NaN,9258.0,NaN,0.979167,5.092690,6.414585,5.224680,...,1.435879,-0.710795,4.240081,1.706246,4.469962,4.790085,4.131412,0.192787,-2.737796,-2.746086


In [16]:
overall_preds = pd.DataFrame(columns=output_cols)

#### National PV & eolien

In [17]:
def df_to_train_test(df):
    # TODO center values
    df_train = df[df.date_cible.dt.year < 2019]
    df_test = df[df.date_cible.dt.year == 2019]
    X_train = df_train.drop(columns=['FC','date_cible'])
    X_test = df_test.drop(columns=['FC','date_cible'])
    y_train = df_train['FC']
    y_test = df_test['FC']
    return df_train,df_test, X_train, X_test, y_train, y_test
def build_term_list(X):
    tl = TermList()
    for i in range(X.shape[1]):
        tl += s(i)
    return tl

for type_prod in ['pv','eolien']:
    _, test_df, X_train, X_test, y_train, _ = df_to_train_test(national_df[national_df.type_prod==type_prod].drop(columns=['perimetre_prod','type_prod','comptage']))
    gam = LinearGAM(build_term_list(X_train)).fit(X_train, y_train)
    test_df['prev_FC'] =gam.predict(X_test)
    test_df['type_prod'] =type_prod
    test_df['perimetre_prod'] ='national'
    overall_preds = pd.concat([overall_preds,test_df[output_cols]], axis=0)

#### Local PV & eolien

In [18]:
local_df = prod[prod.perimetre_prod!='national'].merge(valid_local_meteo_df, on=['date_cible','perimetre_prod'], how='inner')
local_df

,date_cible,perimetre_prod,type_prod,comptage,puissance_installee,FC,time_in_day,date_lancement,tcc,t2m,ssrd,u100,v100,ff100
0,2016-01-02 00:00:00,A.COM,eolien,63.033333,104.7,0.602038,0.000000,2016-01-01 12:00:00,1.000000,6.325450,0.002216,-5.435037,9.539942,10.980536
1,2016-01-02 00:00:00,CXSSE,eolien,57.766667,85.8,0.673271,0.000000,2016-01-01 12:00:00,0.999763,6.039697,0.000753,-4.723014,6.589204,8.123462
2,2016-01-02 00:00:00,HOSTE,pv,0.000000,54.0,0.000000,0.000000,2016-01-01 12:00:00,0.999146,9.673486,-0.002713,4.830697,2.293848,5.401187
3,2016-01-02 00:00:00,ONDES,pv,0.000000,20.9,0.000000,0.000000,2016-01-01 12:00:00,0.998681,10.196870,0.000648,-3.305805,8.543306,9.169785
4,2016-01-02 00:30:00,A.COM,eolien,61.566667,104.7,0.588029,0.020833,2016-01-01 12:00:00,0.999551,6.344030,0.000000,-5.078984,9.433556,10.707797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280221,2019-12-31 22:30:00,ONDES,pv,NaN,26.8,NaN,0.937500,2019-12-30 12:00:00,0.679307,5.737721,0.000000,3.167307,-0.073829,3.218369
280222,2019-12-31 23:00:00,HOSTE,pv,NaN,54.0,NaN,0.958333,2019-12-30 12:00:00,0.845771,8.497054,-0.001417,2.166826,5.074264,5.518028
280223,2019-12-31 23:00:00,ONDES,pv,NaN,26.8,NaN,0.958333,2019-12-30 12:00:00,0.730482,5.867768,-0.002224,3.357581,0.190312,3.400280
280224,2019-12-31 23:30:00,HOSTE,pv,NaN,54.0,NaN,0.979167,2019-12-30 12:00:00,0.917180,8.407278,0.000000,2.244148,4.907121,5.397134


In [19]:
for type_prod in ['pv','eolien']:
    _, test_df, X_train, X_test, y_train, _  = df_to_train_test(local_df[local_df.type_prod==type_prod].drop(columns=['perimetre_prod','type_prod','comptage','date_lancement']))
    gam = LinearGAM(build_term_list(X_train)).fit(X_train, y_train)
    test_df['prev_FC'] =gam.predict(X_test)
    test_df['type_prod'] =type_prod
    test_df['perimetre_prod'] = local_df[(local_df.type_prod==type_prod) & (local_df.date_cible.dt.year ==2019)].perimetre_prod
    overall_preds = pd.concat([overall_preds,test_df[output_cols]], axis=0)

### Postprocessing

In [20]:
# Clip FC
overall_preds.prev_FC = overall_preds.prev_FC.clip(lower=0,upper=1)
# Zero FC at night for pv
overall_preds[(overall_preds.date_cible.dt.hour < 5) \
              & (overall_preds.date_cible.dt.hour >= 23)\
              & (overall_preds.type_prod =='pv')
    ] = 0


overall_preds.date_cible = pd.to_datetime(overall_preds.date_cible)

In [29]:
overall_preds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105026 entries, 105108 to 280129
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   date_cible      105026 non-null  datetime64[ns]
 1   perimetre_prod  105026 non-null  object        
 2   type_prod       105026 non-null  object        
 3   prev_FC         105026 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 4.0+ MB


In [30]:
overall_preds.to_csv('gams_tiy_alexandre_rozier.csv',sep=';', index=False,date_format='%Y-%m-%dT%H:%M:%SZ',float_format="%.3f",decimal='.')

105108   2019-01-01 00:00:00
105110   2019-01-01 00:30:00
105112   2019-01-01 01:00:00
105114   2019-01-01 01:30:00
105116   2019-01-01 02:00:00
                 ...        
280121   2019-12-30 23:00:00
280124   2019-12-30 23:30:00
280125   2019-12-30 23:30:00
280128   2019-12-31 00:00:00
280129   2019-12-31 00:00:00
Name: date_cible, Length: 105026, dtype: datetime64[ns]

# TODO
- Remove PV prod at night
- Center + scale variables
- time in the day

